<a href="https://colab.research.google.com/github/prasadmlexpert/objectidentificationvideo/blob/main/yolov4pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as  np
import cv2

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
ls: cannot access '/content/drive/ImageIdetificationProject': No such file or directory


In [3]:

!ls "/content/drive/MyDrive/ImageIdetificationProject"

'Project Plan'	'Project work'


In [9]:
!ls -l "/content/drive/.short*"

ls: cannot access '/content/drive/.short*': No such file or directory


In [10]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
!ls -l /content/gdrive/MyDrive

ls: /content/gdrive/MyDrive/ImageIdetificationProject: No such file or directory
total 2819
-rw------- 1 root root     143 Nov  5  2021  B18_Titanic.gsheet
drwx------ 2 root root    4096 Nov 28  2021 'Colab Notebooks'
-rw------- 1 root root     258 Jan 29 10:08  Hackthon2.graph
lrw------- 1 root root       0 Jul  2 17:25  ImageIdetificationProject -> '/content/gdrive/MyDrive/Colab Notebooks/ImageIdetificationProject'
-rw------- 1 root root 1847695 Apr  9 09:50  model_combined.pt
-rw------- 1 root root   27389 Nov 12  2021  Numpy.ipynb
-rw------- 1 root root  866685 Jun  1 15:36  picture8378788428864730681.jpg
-rw------- 1 root root     143 Feb 13 08:54  Pre-Hackathon_Alexa_Chatbot.gdoc
-rw------- 1 root root   47483 Apr 16 03:54  PVPrasadGelliSeniorSoftwareEngiLetter.PDF
-rw------- 1 root root   81784 Apr 16 03:53  PVPrasadGelliSeniorSoftwareEngineerAutosarIntegrationResume.PDF
-rw------- 1 root root     360 Dec 29  2021  PythonDataAnalysis.ipynb
-rw------- 1 root root    8480 Feb 19 0